# Investigation of the 2020 Tennessee Outbreak

## Introduction

The first step to any model building, is the investigation of the training data.  For this exercise, we'll be investigating the data around the 2020 Tornado Outbreak which occurred March 2-3, 2020.  One of the five tornadoes in the Nashville area will be investigated.  Google Earth Engine (GEE) and Google Cloud Storage along with OCI's Alyce.ai module will be highlighted in this exploration.  Respectively, these are `ee`, `google.cloud`, and `skai.src.utils` modules.

## Setup

### Environment

For this Jupyter notebook to work, the user must set up the proper environment.  As the demonstration is being conducted in a Vertex AI instance, open Terminal and run the following lines:

```
git clone https://github.com/lanzij-oci/skai.git
pip install -r requirements.txt
```

These steps will clone the github repo where all SKAI code is held and install the proper modules for this exercise.  Note, you should be using a Python 3 interpreter.

### Authentication

Setup Instructions were sent to participants prior to this session outlining registration to Google tools and Google Earth Engine.  

#### Google Cloud Tools Authentication

Individuals will automatically have access to Google tools, e.g. Cloud Storage and BigQuery APIs, and saved resources, e.g. data layers and models, in this workshop's project `gee-workshop-042023`.  In general, users of the Vertex AI Jupyterlab notebook are automatically authenticated with Google.  The expectation is that participants will establish and manage their own google projects going forward.  As such, to authenticate with google resources outside of the Vertex AI platform, it is recommended that users leverage Google's Cloud [SDK](https://cloud.google.com/sdk).

#### Google Earth Engine Authentication

GEE authentication is easy.  Run the following line and follow the necessary steps: 


In [1]:
# Import the google earth engine module and authenticate
import ee
ee.Authenticate()

Enter verification code:  4/1Adeu5BWE17Ds-0zDNzhBodxIfR4dwHiYDg-GQW_fLKgJk0428IZ8B8MUA1w



Successfully saved authorization token.


## Investigation of Pre-Tornado Imagery with GEE


For the SKAI model to be effective, high-resolution RGB images of the earth's surface are required before and after the disaster has taken place.  For this training session, images are sourced from GEE (before) and NOAA (after).  Attendees can access the the NOAA images from [here](https://storms.ngs.noaa.gov/storms/nashville/index.html#16/36.1778/-85.5808).  Documentation on the NAIP imagery can be found [here](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ).

Accessing and visualizing content from GEE [collection](https://developers.google.com/earth-engine/datasets) is highlighted in this section.  Analyses focus on the EF4 tornado that touched down near Cookville, TN.  As part of this section, usage of openstreet maps and visualizing a bounding box will also be highlighted.

### Pulling and Visualizing GEE imagery

In [4]:
# Initializing GEE

ee.Initialize()

In [5]:
nashville_tornado_aoi = ee.Geometry.Polygon([[-87.02166525553643,36.23872528550273],[-87.01933354050459,36.07956369239288],[-85.4159305150307,36.0843353717099],[-85.41498797039785,36.24316124917394],[-87.02166525553643,36.23872528550273]])
tornado_5_aoi = ee.Geometry.Polygon([[-85.68105416273637,36.20591434435648],[-85.68156346863256,36.13942195686942],[-85.50291588726027,36.13839650921801],[-85.50225425962512,36.20487047284015],[-85.68105416273637,36.20591434435648]])

# nashville_tornado_aoi is the area of interest, AOI, corresponding to the region where five tornadoes tracked in the Nashville area
# tornado_5_aoi is the AOI correponding the to the EF4 tornado that will be analyzed in this notebook; subset of nashville_tornado_aoi


In [6]:
# Importing NAIP imagery from GEE

nashville_before = ee.ImageCollection('USDA/NAIP/DOQQ').filterDate('2018-01-01', '2020-02-29').filterBounds(nashville_tornado_aoi)


In [7]:
# Modules for mapping the images in GEE
import folium
from skai.src.utils import add_ee_layer

# Pulling the Openstreet map region where the five tornadoes around Nashville, TN
map = folium.Map(location=tornado_5_aoi.centroid().getInfo()['coordinates'][::-1],zoom_start=13)

# Adding the RGB NAIP layer to the base image
map.add_ee_layer(nashville_before.median(), {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}, 'Nashville, TN Before')

#Displaying the map
display(map)

The region of Tennessee affected by tornadoes is quite large.  To demonstrate the effectiveness of SKAI, the path from one of the tornadoes was selected.  The following sections highlight where this is.

In [8]:
from shapely.geometry import Polygon

folium.GeoJson(Polygon(tornado_5_aoi.getInfo()['coordinates'][0]), 
               name='Tornado #5 - Cookeville, TN',
               style_function=lambda x:{'fillColor': '#FF0000', 'color': '#FF0000', 'fillOpacity': 0.0}).add_to(map)

map.add_child(folium.LayerControl())
display(map)


### SKAI and Openstreet Maps

The artificial intelligence behind SKAI requires the computer to recognize where buildings are.  Many building masks, the 'digitization of buildings,' are available through the OpenStreet Maps module `osmnx`.  However, Facebook's Segment Anythnig Model can be used to automatically identify more buildings or other infrastructure.

In the next boxes of code, masks of buildings is demonstrated.

In [10]:
# Importing the OpenStreet Maps module
import osmnx as ox

# Pulling the polygons of the buildings in the tornado AOI
buildings = ox.features_from_polygon(Polygon(tornado_5_aoi.getInfo()['coordinates'][0]), tags={'building': True}) 
print(f'Total buildings in AOI: {len(buildings)}')

Total buildings in AOI: 9331


In [11]:
# Summarizing types of buildings in the tornado's pathway
buildings['building'].value_counts()

house                 6974
apartments             800
yes                    579
barn                   552
commercial             129
retail                  78
industrial              39
garage                  36
university              35
semidetached_house      27
dormitory               18
school                  15
terrace                 11
church                   8
roof                     6
warehouse                5
hospital                 4
residential              4
construction             3
office                   2
grandstands              2
grandstand               2
shed                     2
Name: building, dtype: int64

In [17]:
# Pulling the Openstreet map region where the five tornadoes around Nashville, TN
map = folium.Map(location=tornado_5_aoi.centroid().getInfo()['coordinates'][::-1],zoom_start=13)

# Adding the RGB NAIP layer to the base image
map.add_ee_layer(nashville_before.median(), {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}, 'Nashville, TN Before')

# Adding the AOI polygon to the map
folium.GeoJson(Polygon(tornado_5_aoi.getInfo()['coordinates'][0]), 
               name='Tornado #5 - Cookeville, TN',
               style_function=lambda x:{'fillColor': '#FF0000', 'color': '#FF0000', 'fillOpacity': 0.0}).add_to(map)

# Adding the building masks to the map
folium.GeoJson(buildings, 
              style_function=lambda x:{'fillColor': '#00FF00', 'color': '#00FF00', 'fillOpacity': 0.3},
              name='Buildings').add_to(map)

map.add_child(folium.LayerControl())
display(map)


## Investigation of Post-Tornado Imagery with Google Cloud Storage


As stated above, NOAA images from [here](https://storms.ngs.noaa.gov/storms/nashville/index.html#16/36.1778/-85.5808).  They are pre-processed, high-resolution images that have alread been downloaded into a Cloud Storage bucket, `skai-tornado` in the Google Cloud Platform project, `gee-workshop-042023`.  Cloud Storage is a good place for users to store non-tabular, geospatial formats such as rasters.  Outside of this workshop, users will need to establish their own projects and buckets.   

In [14]:
# Importing the Cloud module to import post-tornado images
from google.cloud import storage

# Instantiate a GCS client & List all GeoTIFF files in the bucket with the given prefix
storage_client = storage.Client()
blobs = storage_client.list_blobs('skai-tornado', prefix='nashville-images/cog/')
blob_list = [blob.name for blob in blobs if blob.name.endswith('.tif')]

blob_list

['nashville-images/cog/20200307aC0852700w360900n.tif',
 'nashville-images/cog/20200307aC0852700w361030n.tif',
 'nashville-images/cog/20200307aC0852700w361200n.tif',
 'nashville-images/cog/20200307aC0852830w360730n.tif',
 'nashville-images/cog/20200307aC0852830w360900n.tif',
 'nashville-images/cog/20200307aC0852830w361030n.tif',
 'nashville-images/cog/20200307aC0852830w361200n.tif',
 'nashville-images/cog/20200307aC0853000w360730n.tif',
 'nashville-images/cog/20200307aC0853000w360900n.tif',
 'nashville-images/cog/20200307aC0853000w361030n.tif',
 'nashville-images/cog/20200307aC0853000w361200n.tif',
 'nashville-images/cog/20200307aC0853130w360900n.tif',
 'nashville-images/cog/20200307aC0853130w361030n.tif',
 'nashville-images/cog/20200307aC0853130w361200n.tif',
 'nashville-images/cog/20200307aC0853300w361030n.tif',
 'nashville-images/cog/20200307aC0853300w361200n.tif',
 'nashville-images/cog/20200307aC0853430w361030n.tif',
 'nashville-images/cog/20200307aC0853430w361200n.tif',
 'nashvill

The AOI was mosaicked into many raster files.  GEE can easily piece these tifs back into a single image.

In [15]:
# Creating an ImageCollection in EE of the post-tornado RGB image from NOAA
nashville_after = ee.ImageCollection.fromImages([ee.Image.loadGeoTIFF(f'gs://skai-tornado/{path}') for path in blob_list])

For this workshop, we place all of the partitions in the `skai-tornado` bucket.

In [18]:
# Mapping the post-tornado images

# Pulling the Openstreet map region where the five tornadoes around Nashville, TN
map = folium.Map(location=tornado_5_aoi.centroid().getInfo()['coordinates'][::-1],zoom_start=13)

# Adding the post-tornado RGB images
map.add_ee_layer(nashville_after.median(), {'bands': ['B2', 'B1', 'B0'], 'min': 0, 'max': 255}, 'Nashville, TN After')

# Adding the AOI polygon to the map
folium.GeoJson(Polygon(tornado_5_aoi.getInfo()['coordinates'][0]), 
               name='Tornado #5 - Cookeville, TN',
               style_function=lambda x:{'fillColor': '#FF0000', 'color': '#FF0000', 'fillOpacity': 0.0}).add_to(map)

# Adding the building masks to the map
folium.GeoJson(buildings, 
              style_function=lambda x:{'fillColor': '#00FF00', 'color': '#00FF00', 'fillOpacity': 0.3},
              name='Buildings').add_to(map)

map.add_child(folium.LayerControl())
display(map)


With the OpenStreet Map building masks, we can see the devastation of this tornado.

## Model Training

Vertex AI was leveraged in the development of the SKAI model.  [This link](https://github.com/google-research/skai/blob/main/docs/README.md) has instructions of how to develop SKAI for other disasters.  Labelling for this exercise has been completed, and users can access the [interactive labelling process](https://console.cloud.google.com/vertex-ai/locations/us-central1/datasets/8509901140614184960;annotationSetId=985173413603573760/browse?project=gee-workshop-042023).  Subject matter experts typically identify and categorize the severity of damaged buildings.

For the workshop, the Tornado-SKAI model has been built and can be accessed here: `gs://skai-tornado/models/tornado/checkpoints/model.ckpt-00200000`.  When applying the model to new tornado disasters, this is the code:

In [ ]:
from skai.src.utils import run_command
from google.auth import default

# Get the values from environment variables
credentials, project_id = default()
cloud_region = 'us-central1'

# Set the other parameters
params = {
    'project': project_id,
    'location': cloud_region,
    'job_type': 'eval',
    'display_name': 'skai_nashville',                                                                  # change name for costume job
    'eval_docker_image_uri_path': 'gcr.io/disaster-assessment/ssl-eval-uri',
    'service_account': 'skai-427@gee-workshop-042023.iam.gserviceaccount.com',
    'dataset_name': 'skai-nashville',
    'train_dir': 'gs://skai-tornado/models/tornado-nashville/',                                       # change for train dir for predictions -> output at skai-demo/models/refinery_model/predictions/
    'test_examples': 'gs://skai-tornado/nashville/examples/labeled_nashville_buildings.tfrecord',     # change to unlabled data set -- generated above
    'inference_mode': 'True',
    'save_predictions': 'True',
    'eval_ckpt': 'gs://skai-tornado/models/tornado-nashville/checkpoints/model.ckpt-00200384',        # use specific model, not all the model
}

# Generate predictions using params
run_command("src/launch_vertex_job.py",**params)

A TensorFlow environment will need to be spun up for the generation of predictions.